In [ ]:
import os

import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
from glob import glob
from zipfile import ZipFile

import numpy as np

In [ ]:
# Hyperparameter Tunning
input_shape = (32, 32, 3)
num_epochs = 5
batch_size = 16
num_classes = 2

In [ ]:
with ZipFile('../input/aerial-cactus-identification/train.zip','r') as zipObj:
    zipObj.extractall()
train_df = pd.read_csv('../input/aerial-cactus-identification/train.csv', dtype=str)

In [ ]:
train_df.head()

In [ ]:
cat_id = train_df['id']
cat_lbl = train_df['has_cactus']
cat_id
cat_lbl

In [ ]:
id_val = cat_id.values
lbl_val = cat_lbl.values
id_val
lbl_val

In [ ]:
id_val = np.array(id_val)
lbl_val = np.array(lbl_val)
id_val
lbl_val

In [ ]:
classes = [0, 1]

In [ ]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3,3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D((2,2))(net)

net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3,3), padding='SAME')(net)
net = layers.MaxPooling2D((2,2))(net)
net = layers.Dropout(0.25)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(0.5)(net)
net = layers.Dense(2)(net)
net = layers.Activation('softmax')(net)


model = tf.keras.Model(inputs =inputs, outputs=net, name="BaseCNN")

In [ ]:
train_dir = glob('../working/train/')

train_dir

In [ ]:
#tf.data
data_path = train_dir + id_val
data_path[0]

image_raw_tp = tf.io.read_file(data_path[0])
image_tp = tf.io.decode_jpeg(image_raw_tp, )

image_tp.shape

In [ ]:
def get_onehot(lbl_):
    onehot_class = []
    cll = lbl_val[lbl_]
    onehot_class = np.eye(2)[int(cll)]
    return onehot_class

In [ ]:
onehot_classes = []

for lab in range(len(lbl_val)):
    onehot_classes.append(get_onehot(lab))
    
    
onehot_classes

In [ ]:
onehot_class

In [ ]:
def read_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image_raw, )
    return image



In [ ]:
def get_data(path):
    image = read_image(path)
    #print(image)
    onehot = get_onehot(path)   
    #print(onehot)
    return image, onehot

In [ ]:
def get_data_onehot(path, onehot):
    image = read_image(path)
    return image, onehot

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((data_path, onehot_classes))

In [ ]:
dataset = dataset.map(get_data_onehot)

In [ ]:
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(100)
dataset = dataset.repeat()

path, onehot = next(iter(dataset))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model.fit(dataset,
          epochs=3,
          steps_per_epoch=(len(data_path)//batch_size))

In [ ]:
with ZipFile('../input/aerial-cactus-identification/test.zip','r') as zipObj:
    zipObj.extractall()
test_df = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv', dtype=str)


In [ ]:
test_df.head()

In [ ]:
test_id = test_df['id']
#test_lbl = test_df['has_cactus']
test_id_val = np.array(test_id)
#test_lbl_val = np.array(test_lbl)
test_id_val[0]

In [ ]:
test_dir = '../working/test/'
test_path = test_dir + test_id_val
test_path[0]

In [ ]:
teesst = '../working/test/000940378805c44108d287872b2f04ce.jpg'

In [ ]:
def test_read_image(path):
    image_raw = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image_raw, )
    return image
#def test_get_data(path):
#    image = read_image(path)
#    return image

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(test_path)
dataset = dataset.map(test_read_image)
dataset = dataset.batch(1)

In [ ]:
preds = model.predict(dataset)
preds

In [ ]:
result = np.argmax(preds, -1)
result

In [ ]:
test_df['has_cactus'] = result
test_df.head()


test_df.to_csv("../working/submission.csv", index=False)

In [ ]:
#preds_df = pd.read_csv('../working/submission.csv')
#preds_df.head()